In [4]:
conda install -c conda-forge dlib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yyash\anaconda3

  added / updated specs:
Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base -c defaults conda





    - dlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    dlib-19.24.0               |   py39hf8509d4_0         4.4 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following NEW packages will be INSTALLED:

  dlib               conda-forge/win-64::dlib-19.24.0-py39hf8509d4_0
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.12.0-py39haa95532_0 --> conda-forge::conda-4.14.0-py39hcbf5309_0




python_abi-3.9       | 4 KB      |            |   0% 
python_abi-3.9       | 4 KB      | ########## | 100% 
python_abi-3.9       | 4 KB      | ####

In [5]:
!pip install cmake
!pip install face_recognition

  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=e438cb22ab3bfc78f4c789c5838d7cadd94a1678e65ca84c8eb946cf0fa0c1f4
  Stored in directory: c:\users\yyash\appdata\local\pip\cache\wheels\22\a8\60\4a2aeb763d63f50190f4c4e07069a22245347eeafdb3a67551
Successfully built face-recognition-models


In [9]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=5e61aa2edfb7d199ddc94ccf3a163d9f341bcd547fe13757466609c78f29cfd1
  Stored in directory: c:\users\yyash\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [3]:
# Coverage of the face and filter
import dlib
from imutils import face_utils
import os
import numpy as np
import skimage
from scipy.spatial import ConvexHull
import cv2

def cropFacePolygon(image_path, detector, predictor):
    img = dlib.load_rgb_image(image_path)
    # img
    rect = detector(img)[0]
    sp = predictor(img, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])

    outline = landmarks[[*range(17), *range(26,16,-1)]]

    Y, X = skimage.draw.polygon(outline[:,1], outline[:,0])

    cropped_img = np.zeros(img.shape, dtype=np.uint8)
    cropped_img[Y, X] = img[Y, X]

    cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR)
    return cropped_img




detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

for i in range(1, 123):
    src_path = os.path.join("Dataset","Face Dataset", str(i))
    
    src_image_path = os.path.join(src_path, str(i) + ".jpg")
    baseline_img = cropFacePolygon(src_image_path, detector, predictor)
    
    pose_path = os.path.join(src_path, "Filtered")
    cov = []
    for l in range(1, 11):
        
        pose_image_path = os.path.join(pose_path, str(i) + "_" + str(l) + ".jpg")
    
        filtered_img = cropFacePolygon(pose_image_path, detector, predictor)
    
        rows,cols,_ = filtered_img.shape

        dif = 0
        tot = 0
        for j in range(rows):
            for k in range(cols):
                m = abs(np.int64(filtered_img[j, k, 0]) - np.int64(baseline_img[j, k, 0])) + abs(np.int64(filtered_img[j, k, 1]) - np.int64(baseline_img[j, k, 1])) + abs(np.int64(filtered_img[j, k, 2]) - np.int64(baseline_img[j, k, 2]))
                dif = dif + m
                tot += np.int64(baseline_img[j, k, 0]) + np.int64(baseline_img[j, k, 1]) + np.int64(baseline_img[j, k, 2])


        # x100 to get %
        coverage = dif / tot 
#         print(coverage)
        cov.append(coverage)
        print(cov)
    break

[0.03822199249544475]
[0.03822199249544475, 0.23965752183803463]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084, 0.04595638011457292]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084, 0.04595638011457292, 0.241939258193498]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084, 0.04595638011457292, 0.241939258193498, 0.12144863851319508]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084, 0.04595638011457292, 0.241939258193498, 0.12144863851319508, 0.28010783143787144]
[0.03822199249544475, 0.23965752183803463, 0.11925964431302828, 0.13761506053121084, 0.04595638011457292, 0.241939258193498, 0.12144863851319508, 0.28010783143787144, 0.23714202226393435]
[0.03822199249544475, 0.23965752183803463, 0.119259644